In [2]:
import pandas as pd, numpy as np, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
def exponential_kernel(T, alpha):
    """
    Computes exponential kernel weights at time T.
    
    Parameters:
        T (int): Time index to evaluate weights at (typically T = total sample length)
        alpha (float): Exponential decay parameter (0 < alpha < 1)
    
    Returns:
        weights (T,): Kernel weights K_{s,T} for s = 1 to T
    """
    s_vals = np.arange(1, T + 1)
    weights = alpha**(T - s_vals)
    normalized_weights = (1 - alpha) * weights / (1 - alpha**T)
    return normalized_weights

In [4]:
T = 24       # T is the estimation window (the number of months) that we consider in local PCA
alpha = 0.5  # alpha is the exponential decay parameter
exp_decay_weights = exponential_kernel(T, alpha)
print(exp_decay_weights)

[5.96046483e-08 1.19209297e-07 2.38418593e-07 4.76837187e-07
 9.53674373e-07 1.90734875e-06 3.81469749e-06 7.62939499e-06
 1.52587900e-05 3.05175799e-05 6.10351599e-05 1.22070320e-04
 2.44140640e-04 4.88281279e-04 9.76562558e-04 1.95312512e-03
 3.90625023e-03 7.81250047e-03 1.56250009e-02 3.12500019e-02
 6.25000037e-02 1.25000007e-01 2.50000015e-01 5.00000030e-01]


In [5]:

RAW_CSV      = "./Imputed _characteristics.csv"
SAVE_DIR     = Path("./cml_outputs"); SAVE_DIR.mkdir(exist_ok=True)

TRAIN_END    = 201612
OOS_START    = 200001
OOS_END      = 202012 
EPOCHS_DNN   = 3
HIDDEN1, HIDDEN2 = 32, 16
LR           = 1e-4
BATCH_SIZE   = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42); np.random.seed(42)

full_df = pd.read_csv(RAW_CSV)
full_df = full_df.rename(columns={'return': 'Ret'})
# 截断离群点
full_df['Ret'] = full_df['Ret'].clip(lower=full_df['Ret'].quantile(0.01), upper=full_df['Ret'].quantile(0.99))

full_df['date_clean'] = full_df['date'].astype(str).str.split('.').str[0].str.zfill(8)
full_df['yyyymm'] = (
    pd.to_datetime(full_df['date_clean'], format='%Y%m%d')
      .dt.to_period('M')
      .astype(str)
      .str.replace("-", "")
      .astype(int)
)
full_df.drop(columns=['date_clean'], inplace=True)
df = full_df.sort_values(['permno', 'yyyymm']).copy()

feature_cols = df.select_dtypes('number').columns.difference(
    ['permno', 'date', 'yyyymm', 'Ret', 'B2M', 'ME']
).tolist()

lag_cols = [f"{c}_lag" for c in feature_cols]
df[lag_cols] = df.groupby('permno')[feature_cols].shift(1)
lag_features = lag_cols

df['w'] = df['ME'] / df.groupby('yyyymm')['ME'].transform('sum')

df['Ret_next'] = df.groupby('permno')['Ret'].shift(-1)

# 删除为nan值的行
df = df.dropna(subset=['Ret_next', 'w']).copy()

df_valid = df.dropna(subset=['Ret_next', 'w']).copy()

index_returns = (
    df_valid.groupby('yyyymm')
            .apply(lambda g: np.sum((g['w'] / g['w'].sum()) * g['Ret_next']))
            .reset_index().rename(columns={0: 'y_next'})
)
index_returns.to_csv(SAVE_DIR / "index_returns_full.csv", index=False)

/tmp/ipykernel_1131/3167354070.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.sum((g['w'] / g['w'].sum()) * g['Ret_next']))


In [6]:
# I–III
train_df = df[df['yyyymm'] <= TRAIN_END].copy()
g_list_train = []
predicted_returns = []
t = 0
all_xhat_list = []   # Save (permno, yyyymm, predicted_x)
all_beta_dicts = []   # Save (permno, yyyymm, beta_1…beta_K_g)
all_g_dicts = []      # Save (yyyymm, g_1…g_K_g)
K_g = 5
G_train = []
# pca_file = 'pca_outputs/'
# beta_records = []
for m in tqdm(sorted(train_df['yyyymm'].unique()), desc="Months"):
    dm = train_df[(train_df['yyyymm'] == m)].dropna(subset=['Ret_next'] + lag_features)
    if dm.empty:
        continue

    # Step I: DNN 
    X = dm[lag_features].values; y = dm['Ret_next'].values
    scaler = StandardScaler().fit(X)
    X_scaled = scaler.transform(X)
    X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y, dtype=torch.float32).view(-1,1).to(device)

    net = nn.Sequential(
        nn.Linear(X.shape[1], HIDDEN1), nn.ReLU(),
        nn.Linear(HIDDEN1, HIDDEN2), nn.ReLU(),
        nn.Linear(HIDDEN2, 1)
    ).to(device)
    opt = optim.Adam(net.parameters(), lr=LR)
    loss_fn = nn.MSELoss()

    net.train()
    loader = DataLoader(TensorDataset(X_tensor, y_tensor),
                        batch_size=BATCH_SIZE, shuffle=True)

    for epoch in range(EPOCHS_DNN):
        for xb, yb in loader:
            opt.zero_grad()
            loss = loss_fn(net(xb), yb)
            loss.backward()
            opt.step()

    net.eval()
    with torch.no_grad():
        preds = net(X_tensor).squeeze().cpu().numpy()
        df_month = dm.copy()
        df_month['predicted_x'] = preds
        predicted_returns.append(df_month[['permno', 'yyyymm', 'Ret_next', 'predicted_x'] + list(feature_cols)])
        
    # xhat_df = df_month[['permno', 'yyyymm', 'predicted_x', 'Ret_next']].copy()
    # all_xhat_list.append(xhat_df)
    
    # step 2 local PCA
    t_start = max(0, t - T + 1)
    t += 1
    df_window = predicted_returns[t_start:(t+1)]
    num_ts = len(df_window)
    weights = exp_decay_weights[-num_ts:]
    w = weights / weights.sum()

    # Merge all months in the window to form X matrix 
    # df_months = [df[['permno', 'predicted_x']] for df in df_window]

    # Rename prevent join error
    df_months = [
    df_t[['permno', 'predicted_x']].rename(columns={'predicted_x': f'pred_{i}'})
    for i, df_t in enumerate(df_window)
]
    X_df = df_months[0]
    for df_t in df_months[1:]:
        X_df = pd.merge(X_df, df_t, on='permno')
    
    permnos = X_df['permno'].values
    X_mat = X_df.drop(columns=['permno']).values # shape: N x num_ts

    # =============== PCA ===============
    # Center across time using weighted mean
    X_mean = np.average(X_mat, axis=1, weights=w)[:, None]
    X_centered = X_mat - X_mean

    # Compute weighted cross-sectional covariance: Σ = X W X^T
    X_weighted = X_centered * np.sqrt(w)[None, :]
    X_cov = X_weighted @ X_weighted.T  # N x N

    # Eigendecomposition
    eigvals, eigvecs = np.linalg.eigh(X_cov)
    idx = np.argsort(eigvals)[::-1]  # descending order
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    # Top K_g eigenvectors: asset loadings ("betas")
    components = eigvecs[:, :K_g]  # N x K_g
    for i, p in enumerate(permnos):
        all_beta_dicts.append({
            'permno': p,
            'yyyymm': m,
            'beta': components[i, :].copy()  # Length K_g NumPy array
        })
    v = np.array(pd.merge(dm, X_df, on='permno')['B2M']) # Length N vector
    g = (components.T @ v) / len(v) # g K_g vector
    G_train.append({'yyyymm': m, 'g_t': g})

Months: 100%|██████████| 480/480 [29:37<00:00,  3.70s/it]


### 分为两个表，一个是 
1. permno, yyyymm, x_hat, Ret_next
2. yyyymm, g_t, y_t

In [7]:
xhat_all = pd.concat(predicted_returns, ignore_index=True)
xhat_all = xhat_all[['permno', 'yyyymm', 'Ret_next', 'predicted_x']]
beta_all = pd.DataFrame(all_beta_dicts)


In [8]:
merged_px_beta = pd.merge(
    xhat_all,
    beta_all,
    on=['permno', 'yyyymm'],
    how='inner'
)

In [9]:
# In‐Sample x_hat
merged_px_beta.to_csv(SAVE_DIR/"xhat_train.csv", index=False)

gdf_train = pd.DataFrame(G_train)
gdf_train.to_csv(SAVE_DIR/"g_train.csv", index=False)

In [10]:
# ==================== STEP 3: IN‐SAMPLE OLS ====================

idx_ret_train = index_returns[index_returns['yyyymm'] <= TRAIN_END]
reg_train = pd.merge(gdf_train, idx_ret_train, on='yyyymm', how='inner').dropna()

X_tr = sm.add_constant(reg_train['g_t'].tolist())
Y_tr = reg_train['y_next']
ols = sm.OLS(Y_tr, X_tr).fit(cov_type='HAC', cov_kwds={'maxlags':3})
print("\n=== In‐Sample OLS Summary ===")
print(ols.summary())



=== In‐Sample OLS Summary ===
                            OLS Regression Results                            
Dep. Variable:                 y_next   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.083
Date:                Sun, 08 Jun 2025   Prob (F-statistic):              0.369
Time:                        03:23:34   Log-Likelihood:                -911.60
No. Observations:                 479   AIC:                             1835.
Df Residuals:                     473   BIC:                             1860.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1432

In [11]:
c = np.array(ols.params)

In [12]:
results = []
idx_ret_full = index_returns.set_index('yyyymm')
predicted_returns = []

xhat_oos_list = []
oos_train_df = df[(df['yyyymm'] <= OOS_END) & (df['yyyymm'] >= OOS_START)].copy()
g_list_train = []
predicted_returns = []
t = 0
all_xhat_list = []   # Save (permno, yyyymm, predicted_x)
all_beta_dicts = []   # Save (permno, yyyymm, beta_1…beta_K_g)
all_g_dicts = []      # Save (yyyymm, g_1…g_K_g)
K_g = 5
G_train = []
# pca_file = 'pca_outputs/'
# beta_records = []
for m in tqdm(sorted(oos_train_df['yyyymm'].unique()), desc="Months"):
    if m < OOS_START or m > OOS_END:
        continue
    df_past = oos_train_df[oos_train_df['yyyymm'] <= m].copy()
    dm = df_past[df_past['yyyymm'] == m].dropna(subset=['Ret'] + lag_features)
    if dm.empty:
        continue
    # print("****")
    # Step I: DNN 回归
    hist = df_past.dropna(subset=['Ret'] + lag_features)
    X_hist = hist[lag_features].values
    y_hist = hist['Ret'].values

    scaler_m = StandardScaler().fit(X_hist)
    X_hist_scaled = scaler_m.transform(X_hist)
    X_hist_tensor = torch.tensor(X_hist_scaled, dtype=torch.float32).to(device)
    y_hist_tensor = torch.tensor(y_hist, dtype=torch.float32).view(-1,1).to(device)
    net_m = nn.Sequential(
        nn.Linear(X.shape[1], HIDDEN1), nn.ReLU(),
        nn.Linear(HIDDEN1, HIDDEN2), nn.ReLU(),
        nn.Linear(HIDDEN2, 1)
    ).to(device)
    opt_m = optim.Adam(net_m.parameters(), lr=LR)
    loss_fn = nn.MSELoss()

    net_m.train()
    loader_m = DataLoader(TensorDataset(X_hist_tensor, y_hist_tensor),
                          batch_size=BATCH_SIZE, shuffle=True)
    for epoch in range(EPOCHS_DNN):
        for xb, yb in loader_m:
            opt_m.zero_grad()
            loss = loss_fn(net_m(xb), yb)
            loss.backward()
            opt_m.step()

    # Predict x_hat for month m
    X_m = dm[lag_features].values
    X_m_scaled = scaler_m.transform(X_m)
    X_m_tensor = torch.tensor(X_m_scaled, dtype=torch.float32).to(device)
    net_m.eval()
    with torch.no_grad():
        x_hat_m = net_m(X_m_tensor).squeeze().cpu().numpy()
        df_month = dm.copy()
        df_month['predicted_x'] = x_hat_m
        predicted_returns.append(df_month[['permno', 'yyyymm', 'Ret_next', 'predicted_x'] + list(feature_cols)])
        
    # ——— 新增：把当月的 OOS x_hat_m 及对应 permno、yyyymm 存到列表里 ———
    tmp_oos = dm[['permno']].reset_index(drop=True).copy()
    tmp_oos['yyyymm'] = m
    tmp_oos['x_hat'] = x_hat_m
    xhat_oos_list.append(tmp_oos)
    # xhat_df = df_month[['permno', 'yyyymm', 'predicted_x', 'Ret_next']].copy()
    # all_xhat_list.append(xhat_df)
    
    # step 2 local PCA
    t_start = max(0, t - T + 1)
    t += 1
    df_window = predicted_returns[t_start:(t+1)]
    num_ts = len(df_window)
    weights = exp_decay_weights[-num_ts:]
    w = weights / weights.sum()

    # Merge all months in the window to form X matrix 
    # df_months = [df[['permno', 'predicted_x']] for df in df_window]

    # 重命名避免join时候出错
    df_months = [
    df[['permno', 'predicted_x']].rename(columns={'predicted_x': f'pred_{i}'})
    for i, df in enumerate(df_window)
]
    X_df = df_months[0]
    for df_ in df_months[1:]:
        X_df = pd.merge(X_df, df_, on='permno')
    
    permnos = X_df['permno'].values
    X_mat = X_df.drop(columns=['permno']).values # shape: N x num_ts

    # =============== PCA ===============
    # Center across time using weighted mean
    X_mean = np.average(X_mat, axis=1, weights=w)[:, None]
    X_centered = X_mat - X_mean

    # Compute weighted cross-sectional covariance: Σ = X W X^T
    X_weighted = X_centered * np.sqrt(w)[None, :]
    X_cov = X_weighted @ X_weighted.T  # N x N

    # Eigendecomposition
    eigvals, eigvecs = np.linalg.eigh(X_cov)
    idx = np.argsort(eigvals)[::-1]  # descending order
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    # Top K_g eigenvectors: asset loadings ("betas")
    components = eigvecs[:, :K_g]  # N x K_g
    for i, p in enumerate(permnos):
        all_beta_dicts.append({
            'permno': p,
            'yyyymm': m,
            'beta': components[i, :].copy()  # Length K_g NumPy array
        })
    v = np.array(pd.merge(dm, X_df, on='permno')['B2M']) # N
    g = (components.T @ v) / len(v) # g is K_g vector
    G_train.append({'yyyymm': m, 'g_t': g})


    y_hat = np.insert(g, 0, 1) @ c.T

    # # True y_{m+1}
    if m not in idx_ret_full.index:
        continue
    y_true = float(idx_ret_full.loc[m, 'y_next'])

    results.append({
        'yyyymm': m,
        'g_t': g,
        'y_true': y_true,
        'y_hat': y_hat 
    })


Months: 100%|██████████| 251/251 [3:20:46<00:00, 48.00s/it]  


In [13]:
# Save All OOS x_hat
all_xhat_oos = pd.concat(xhat_oos_list, ignore_index=True)
all_xhat_oos.to_csv(SAVE_DIR/"xhat_oos.csv", index=False)

# Save OOS prediction result
res_df = pd.DataFrame(results)
res_df.to_csv(SAVE_DIR/"oos_results.csv", index=False)
print("   → oos_results.csv saved.")

# Compute OOS metric
mse = np.mean((res_df['y_true'] - res_df['y_hat'])**2)
y_bar = Y_tr.mean()
mse_bench = np.mean((res_df['y_true'] - y_bar)**2)
r2_oos = 1 - mse / mse_bench
hit  = np.mean(np.sign(res_df['y_true']) == np.sign(res_df['y_hat']))

print(f"\n=== OOS Validation Set({OOS_START}–{OOS_END}) ===")
print(f"OOS-MSE   : {mse:.6e}")
print(f"OOS-R²    : {r2_oos:.4f}")
print(f"Hit-Rate  : {hit:.4f}")

   → oos_results.csv saved.

=== OOS Validation Set(200001–202012) ===
OOS-MSE   : 4.848314e+00
OOS-R²    : -0.0136
Hit-Rate  : 0.5857
